## Assignment 1.1 

### Sindhu Bhattarai



## ADS 509 Module 1: APIs and Web Scraping

**Instruction:**

This notebook has three parts. In the first part you will pull data from the Twitter API. In the second, you will scrape lyrics from AZLyrics.com. In the last part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 100,000 Twitter followers and 20 songs with lyrics on AZLyrics.com. In this part of the assignment we pull the some of the user information for the followers of your artist and store them in text files. 


In [1]:
pip show tweepy

Name: tweepy
Version: 4.10.1
Summary: Twitter library for Python
Home-page: https://www.tweepy.org/
Author: Joshua Roesslein
Author-email: tweepy@googlegroups.com
License: MIT
Location: /Users/sakshyamdahal/opt/anaconda3/lib/python3.7/site-packages
Requires: requests, requests-oauthlib, oauthlib
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Twitter API Pull

In [2]:
# for the twitter section
import tweepy
import os
import datetime
import re
from pprint import pprint

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter


**Instruction:**

We need bring in our API keys. Since API keys should be kept secret, we'll keep them in a file called `api_keys.py`. This file should be stored in the directory where you store this notebook. The example file is provided for you on Blackboard. The example has API keys that are _not_ functional, so you'll need to get Twitter credentials and replace the placeholder keys. 

In [3]:
#bringing the api keys 

from api_keys import api_key, api_key_secret, bearer_token


In [4]:
client = tweepy.Client(bearer_token,wait_on_rate_limit=True)

# Testing the API

The Twitter APIs are quite rich. Let's play around with some of the features before we dive into this section of the assignment. For our testing, it's convenient to have a small data set to play with. We will seed the code with the handle of John Chandler, one of the instructors in this course. His handle is `@37chandler`. Feel free to use a different handle if you would like to look at someone else's data. 

We will write code to explore a few aspects of the API: 

1. Pull some of the followers @37chandler.
1. Explore response data, which gives us information about Twitter users. 
1. Pull the last few tweets by @37chandler.


In [5]:
handle = "37chandler"
user_obj = client.get_user(username=handle)

followers = client.get_users_followers(
    # Learn about user fields here: 
    # https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
    user_obj.data.id, user_fields=["created_at","description","location",
                                   "public_metrics"]
)



Now let's explore these a bit. We'll start by printing out names, locations, following count, and followers count for these users. 

In [6]:
num_to_print = 20

for idx, user in enumerate(followers.data) :
    following_count = user.public_metrics['following_count']
    followers_count = user.public_metrics['followers_count']
    
    print(f"{user.name} lists '{user.location}' as their location.")
    print(f" Following: {following_count}, Followers: {followers_count}.")
    print()
    
    if idx >= (num_to_print - 1) :
        break
    

Dave Renn lists 'None' as their location.
 Following: 42, Followers: 10.

Lionel lists 'None' as their location.
 Following: 202, Followers: 204.

Megan Randall lists 'None' as their location.
 Following: 141, Followers: 100.

Jacob Salzman lists 'None' as their location.
 Following: 562, Followers: 134.

twiter not fun lists 'None' as their location.
 Following: 221, Followers: 21.

Hariettwilsonincarnate lists 'None' as their location.
 Following: 218, Followers: 60.

Christian Tinsley lists 'None' as their location.
 Following: 2, Followers: 0.

Steve lists 'I'm over here.' as their location.
 Following: 1592, Followers: 33.

John O'Connor 🇺🇦 lists 'None' as their location.
 Following: 8, Followers: 1.

CodeGrade lists 'Amsterdam' as their location.
 Following: 2820, Followers: 424.

Cleverhood lists 'Providence, RI' as their location.
 Following: 2795, Followers: 3562.

Regina 🚶‍♀️🚲🌳 lists 'Minneapolis' as their location.
 Following: 2802, Followers: 3336.

Eric Hallstrom lists 'Mi

Let's find the person who follows this handle who has the most followers. 

In [7]:
max_followers = 0

for idx, user in enumerate(followers.data) :
    followers_count = user.public_metrics['followers_count']
    
    if followers_count > max_followers :
        max_followers = followers_count
        max_follower_user = user

        
print(max_follower_user)
print(max_follower_user.public_metrics)

WedgeLIVE
{'followers_count': 14180, 'following_count': 2222, 'tweet_count': 56103, 'listed_count': 218}


Let's pull some more user fields and take a look at them. The fields can be specified in the `user_fields` argument. 

In [8]:
response = client.get_user(id=user_obj.data.id,
                          user_fields=["created_at","description","location",
                                       "entities","name","pinned_tweet_id","profile_image_url",
                                       "verified","public_metrics"])

In [9]:
for field, value in response.data.items() :
    print(f"for {field} we have {value}")

for public_metrics we have {'followers_count': 193, 'following_count': 589, 'tweet_count': 997, 'listed_count': 3}
for description we have He/Him. Data scientist, urban cyclist, educator, erstwhile frisbee player. 

¯\_(ツ)_/¯
for id we have 33029025
for created_at we have 2009-04-18 22:08:22+00:00
for verified we have False
for name we have John Chandler
for location we have MN
for profile_image_url we have https://pbs.twimg.com/profile_images/2680483898/b30ae76f909352dbae5e371fb1c27454_normal.png
for username we have 37chandler


### **Questions**

**Now a few questions for you about the user object.**

Q: How many fields are being returned in the `response` object? 

A:  Nine (9) fields are used in the response object.  

---

Q: Are any of the fields within the user object non-scalar? (I.e., more complicated than a simple data type like integer, float, string, boolean, etc.) 

A: Yes, the feature public_metrics is in format dictionary. 

---

Q: How many friends, followers, and tweets does this user have? 

A: This user have 193 followers, 589 whom he is following and 996 tweets. 


Although you won't need it for this assignment, individual tweets can be a rich source of text-based data. To illustrate the concepts, let's look at the last few tweets for this user. You are encouraged to explore the fields that are available about Tweets.

In [10]:

response = client.get_users_tweets(user_obj.data.id)

# By default, only the ID and text fields of each Tweet will be returned
for idx, tweet in enumerate(response.data) :
    print(tweet.id)
    print(tweet.text)
    print()
    
    if idx > 10 :
        break

1569760631548690437
RT @dtmooreeditor: So there's a particular quirk of English grammar that I've always found quite endearing: the exocentric verb-noun compou…

1569155273742327811
As a Minneapolis person, I knew we had Toronto beat, but I didn't realize Portland had us beat: https://t.co/xrx5mOFcWK.

But @nytimes, c'mon! https://t.co/M9mBWhdgsj

1568982292923826176
RT @wonderofscience: Amazing lenticular cloud over Mount Fuji

Credit: Iurie Belegurschi
https://t.co/0mUxl28H9U

1568242374085869570
RT @depthsofwiki: lots of memes about speedy wikipedia editors — quick thread about what went down on wikipedia in the minutes after her de…

1568074978754703361
@DrLaurenWilson @leighradwood @MaritsaGeorgiou @Walgreens I could not possibly agree more with this sentiment. Compared to almost any other primary care I've received, they are great.

1567530169686196224
@DrLaurenWilson @MaritsaGeorgiou @Walgreens For those who have access to Curry Health Center on campus, you can get a bivalent bo

## Pulling Follower Information

**Instructions:**

In this next section of the assignment, we will pull information about the followers of your two artists. We've seen above how to pull a set of followers using `client.get_users_followers`. This function has a parameter, `max_results`, that we can use to change the number of followers that we pull. Unfortunately, we can only pull 1000 followers at a time, which means we will need to handle the _pagination_ of our results. 

The return object has the `.data` field, where the results will be found. It also has `.meta`, which we use to select the next "page" in the results using the `next_token` result. I will illustrate the ideas using our user from above. 


### Rate Limiting

Twitter limits the rates at which we can pull data, as detailed in [this guide](https://developer.twitter.com/en/docs/twitter-api/rate-limits). We can make 15 user requests per 15 minutes, meaning that we can pull $4 \cdot 15 \cdot 1000 = 60000$ users per hour. I illustrate the handling of rate limiting below, though whether or not you hit that part of the code depends on your value of `handle`.  


In the below example, I'll pull all the followers, 25 at a time. (We're using 25 to illustrate the idea; when you do this set the value to 1000.) 

In [11]:
handle_followers = []
pulls = 0
max_pulls = 100
next_token = None

while True :

    followers = client.get_users_followers(
        user_obj.data.id, 
        max_results=25, # when you do this for real, set this to 1000!
        pagination_token = next_token,
        user_fields=["created_at","description","location",
                     "entities","name","pinned_tweet_id","profile_image_url",
                     "verified","public_metrics"]
    )
    pulls += 1
    
    for follower in followers.data : 
        follower_row = (follower.id,follower.name,follower.created_at,follower.description)
        handle_followers.append(follower_row)
    
    if 'next_token' in followers.meta and pulls < max_pulls :
        next_token = followers.meta['next_token']
    else : 
        break
        


## Pulling Twitter Data for Your Artists

Now let's take a look at your artists and see how long it is going to take to pull all their followers. 

In [12]:
## checking the time to pull the information of the artists I like

artists = dict()

for handle in ['selenagomez','Adele'] : 
    user_obj = client.get_user(username=handle,user_fields=["public_metrics"])
    artists[handle] = (user_obj.data.id, 
                       handle,
                       user_obj.data.public_metrics['followers_count'])
    

for artist, data in artists.items() : 
    print(f"It would take {data[2]/(1000*15*4):.2f} hours to pull all {data[2]} followers for {artist}. ")
    

It would take 1105.00 hours to pull all 66300064 followers for selenagomez. 
It would take 461.14 hours to pull all 27668170 followers for Adele. 


In [14]:
# Make the "twitter" folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then "unlink" it. Then create a new one.

if not os.path.isdir("twitter") : 
    #shutil.rmtree("twitter/")
    os.mkdir("twitter")

**Instruction:**

In this following cells, build on the above code to pull some of the followers and their data for your two artists. As you pull the data, write the follower ids to a file called `[artist name]_followers.txt` in the "twitter" folder. For instance, for Cher I would create a file named `cher_followers.txt`. As you pull the data, also store it in an object like a list or a data frame.

In addition to creating a file that only has follower IDs in it, you will create a file that includes user data. From the response object please extract and store the following fields: 

* screen_name	
* name	
* id	
* location	
* followers_count	
* friends_count	
* description

Store the fields with one user per row in a tab-delimited text file with the name `[artist name]_follower_data.txt`. For instance, for Cher I would create a file named `cher_follower_data.txt`. 

One note: the user's description can have tabs or returns in it, so make sure to clean those out of the description before writing them to the file. I've included some example code to do that below the stub. 

In [15]:
num_followers_to_pull = 100*1000 # feel free to use this to limit the number of followers you pull.

In [16]:
# Modify the below code stub to pull the follower IDs and write them to a file. 

handles = ['selenagomez','Adele']

whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()


# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

path = 'twitter'

for handle in handles :
    
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs

     # Create the output file names

    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_followers_data.txt"
    
    
    print(f'pulling followers for{handle}.')
    user_obj = client.get_user(username = handle)
    print(user_obj)
    
    #path to store the extracted data
    followers_output_file_path = os.path.join(path,followers_output_file)
    user_data_output_file_path = os.path.join(path,user_data_output_file)
    
# Using tweepy.Paginator (https://docs.tweepy.org/en/latest/v2_pagination.html), 
# use `get_users_followers` to pull the follower data requested. 
    
    for followers in tweepy.Paginator(client.get_users_followers,user_obj.data.id,  
                                      user_fields=["description","location","public_metrics"],
                                      max_results=1000, limit=101):
      # For each response object, extract the needed fields and store them in a dictionary or
    # data frame. 
        for i in followers.data:
            followers_data[handle].append(i.id)
            
        
            user_data_rows = [i.username,
                            i.name,
                             i.id,
                         i.location,
                             i.public_metrics['followers_count'],
                             i.public_metrics['following_count'],
                             re.sub(r"\s+","",i.description).strip()]
            user_data[handle].append(user_data_rows)

    # If you've pulled num_followers_to_pull, feel free to break out paged twitter API response


        if len(followers)>= num_followers_to_pull:
            break
            
        print(f'finished pulling followers for {handle}.')
        
        #writting the data in the working directory for individual artists
        with open(followers_output_file_path, 'w') as f1: 
            for follower_id in followers_data[handle]: 
                f1.write('%s\n' % (follower_id))
                
                
        with open(user_data_output_file_path, 'w') as f2: 
            f2.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format('screen_name', 'name', 'id', 'location',
                                                        'followers_count', 'friends_count', 'description'))
            for row in user_data[handle]: 
                for i in range(len(row)):
                    if i == len(row) - 1:
                        f2.write('%s\n' % (row[i]))
                    else:
                        f2.write('%s\t' % (row[i]))

        
# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)


pulling followers forselenagomez.
Response(data=<User id=23375688 name=Selena Gomez username=selenagomez>, includes={}, errors=[], meta={})
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.


Rate limit exceeded. Sleeping for 703 seconds.


finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.


Rate limit exceeded. Sleeping for 895 seconds.


finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.
finished pulling followers for selenagomez.


...

finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.


Rate limit exceeded. Sleeping for 889 seconds.


finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.
finished pulling followers for Adele.


Rate limit exceeded. Sleeping for 882 seconds.


finished pulling followers for Adele.
3:27:53.529902


---

# Lyrics Scrape

**Question:**

This section asks you to pull data from the Twitter API and scrape www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [17]:
# we'll use this dictionary to hold both the artist name and the link on AZlyrics
artists = {'selena':"https://www.azlyrics.com/s/selenagomez.html",
           'adele':"https://www.azlyrics.com/a/adele.html"} 


## A Note on Rate Limiting

**Instructions:**

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

**Question:**

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A:
User-agent: *
Disallow: /lyricsdb/
Disallow: /song/
Allow: /

User-agent: 008
Disallow: /


The first part means restrict all search engine from crawling the /lyricsdb and /song/ directory on the site and everything within it (subdirectories). Rest all directory are allowed to crawl.

The second part means it would block OO8 that “listen” to robots.txt from crawling the site.

The webscraping we are about to to is allowed by the page. I know it by reading regular expression of user-agent.

In [18]:
import random
from bs4 import BeautifulSoup

# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())
    
    # now extract the links to lyrics pages from this page
    soup = BeautifulSoup(r.content)
    links = soup.select('a[href^="/lyrics/"]')
   
 # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links.
    lyrics_pages[artist] = ["https://www.azlyrics.com" + i.get("href") for i in links]
    
    

Let's make sure we have enough lyrics pages to scrape. 

In [19]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [20]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For selena we have 95.
The full pull will take for this artist will take 0.26 hours.
For adele we have 71.
The full pull will take for this artist will take 0.2 hours.


## Part 2: Pulling Lyrics

**Instruction:**

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
2. Iterate over the artists in `lyrics_pages`. 
3. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
4. Iterate over the pages. 
5. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
6. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [21]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [22]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

import shutil

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [23]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist in lyrics_pages :

    # Use this space to carry out the following steps: 
    
# 1. Build a subfolder for the artist
 # 2. Iterate over the lyrics pages
    
    path = 'lyrics'
    
    artist_file_path = os.path.join(path,"{}".format(artist))

    if not os.path.exists(artist_file_path):
        os.mkdir(artist_file_path)
 
 # 3. Request the lyrics page.
# Don’t forget to add a line like `time.sleep(5 + 10*random.random()) to sleep after making the request
    for music in lyrics_pages[artist]:
        
        r = requests.get(music)
        
        time.sleep(5 + 10*random.random())
        
        lyrics_soup = BeautifulSoup(r.content)
    
    
# 4. Extract the title and lyrics from the page.
        initial = lyrics_soup.find('b')
        
        title = initial.find_next('b').get_text()
        
        lyrics = lyrics_soup.find('div', class_=False, id=False).get_text()
   
 # 5. Write out the title, two returns (‘\n’), and the lyrics. Use `generate_filename_from_url to generate the filename.
        
        lyrics_title = title + '\n' + '\n' + lyrics
        
        open_path = os.path.join(artist_file_path, generate_filename_from_link(music))
        
        file = open(open_path,'w')
        
        file.write(lyrics_title)
    
    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist


  

In [24]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.61 hours.


---

# Evaluation

This assignment asks you to pull data from the Twitter API and scrape www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [25]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

---

## Checking Twitter Data

The output from your Twitter API pull should be two files per artist, stored in files with formats like `cher_followers.txt` (a list of all follower IDs you pulled) and `cher_followers_data.txt`. These files should be in a folder named `twitter` within the repository directory. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [26]:
twitter_files = os.listdir("twitter")
twitter_files = [f for f in twitter_files if f != ".DS_Store"]
artist_handles = list(set([name.split("_")[0] for name in twitter_files]))

print(f"We see two artist handles: {artist_handles[0]} and {artist_handles[1]}.")

We see two artist handles: selenagomez and Adele.


In [27]:
for artist in artist_handles :
    follower_file = artist + "_followers.txt"
    follower_data_file = artist + "_followers_data.txt"
    
    ids = open("twitter/" + follower_file,'r').readlines()
    
    print(f"We see {len(ids)-1} in your follower file for {artist}, assuming a header row.")
    
    with open("twitter/" + follower_data_file,'r') as infile :
        
        # check the headers
        headers = infile.readline().split("\t")
        
        print(f"In the follower data file ({follower_data_file}) for {artist}, we have these columns:")
        print(" : ".join(headers))
        
        description_words = []
        locations = set()
        
        
        for idx, line in enumerate(infile.readlines()) :
            line = line.strip("\n").split("\t")
            
            try : 
                locations.add(line[3])            
                description_words.extend(words(line[6]))
            except :
                pass
    
        

        print(f"We have {idx+1} data rows for {artist} in the follower data file.")

        print(f"For {artist} we have {len(locations)} unique locations.")

        print(f"For {artist} we have {len(description_words)} words in the descriptions.")
        print("Here are the five most common words:")
        print(Counter(description_words).most_common(5))

        
        print("")
        print("-"*40)
        print("")

We see 100985 in your follower file for selenagomez, assuming a header row.
In the follower data file (selenagomez_followers_data.txt) for selenagomez, we have these columns:
screen_name : name : id : location : followers_count : friends_count : description

We have 101006 data rows for selenagomez in the follower data file.
For selenagomez we have 7112 unique locations.
For selenagomez we have 58420 words in the descriptions.
Here are the five most common words:
[('i', 767), ('t', 615), ('co', 574), ('https', 370), ('she', 289)]

----------------------------------------

We see 100956 in your follower file for Adele, assuming a header row.
In the follower data file (Adele_followers_data.txt) for Adele, we have these columns:
screen_name : name : id : location : followers_count : friends_count : description

We have 100976 data rows for Adele in the follower data file.
For Adele we have 12380 unique locations.
For Adele we have 121943 words in the descriptions.
Here are the five most c

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [28]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For selena we have 92 files.
For selena we have roughly 34712 words, 2374 are unique.
For adele we have 71 files.
For adele we have roughly 22315 words, 1579 are unique.
